# WORK IN PROGRESS ! 

## FOR DATASET https://www.kaggle.com/awsaf49/happywhale-cropped-dataset-yolov5 thanks @awsaf49


In [ ]:
# #For TPU
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version 1.7 --apt-packages libomp5 libopenblas-dev
# ! pip -q install pytorch-lightning==1.1.5

In [ ]:
# importing required stuff
# !pip -q install --upgrade wandb
import pandas as pd
import albumentations
import torchvision 
from PIL import Image
import wandb
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import torch 
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb.login(key = user_secrets.get_secret("wandb_key"))
import cv2
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl 
import torch 
import numpy as np 
import pandas as pd
import  os 
import sys 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master")
import timm
import math
import matplotlib.pyplot as plt
import torchvision

In [ ]:
!unzip -q ../input/happywhale-cropped-dataset-yolov5/train_images.zip
!unzip -q  ../input/happywhale-cropped-dataset-yolov5/test_images.zip
train = pd.read_csv("../input/happywhale-cropped-dataset-yolov5/train.csv")
test = pd.read_csv("../input/happywhale-cropped-dataset-yolov5/test.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train.species= le.fit_transform(train.species)

In [ ]:
DIFF_SPECIES = le.classes_

In [ ]:
# train.image_id = train.image_id.apply(lambda x: f"./train_images/{x}")

In [ ]:


# run = wandb.init(project = "hyperwhaletpu",name = "pytorchlightning")
BATCH_SIZE = 8
ss = pd.read_csv("../input/happy-whale-and-dolphin/sample_submission.csv")
train['image_id'] = train["image_id"].apply(lambda x: f"./train_images/{x}")
ss['image'] = ss['image'].apply(lambda x : f"./test_images/{x}")

landmark_id2idx = {landmark_id: idx for idx, landmark_id in enumerate(sorted(train['individual_id'].unique()))}
idx2landmark_id = {idx: landmark_id for idx, landmark_id in enumerate(sorted(train['individual_id'].unique()))}
train['individual_id'] = train['individual_id'].map(landmark_id2idx)

# train = train.sample(frac = 0.001)
train,val = train_test_split(train)

In [ ]:
N_CLASSES= len(landmark_id2idx)

In [ ]:
# transforms_224= albumentations.Compose([
#     albumentations.Resize(224,224),
#     albumentations.Normalize()
# ])

# transforms_768 = albumentations.Compose([
#     albumentations.Resize(768, 768),
#     albumentations.Normalize()  
# ])
transforms_512 = albumentations.Compose([
    albumentations.Resize(512, 512),
#     albumentations.Normalize()
])

In [ ]:
# creating the pytroch model AlexModel 
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, file_paths , labels , transforms, species= None, is_test = False):
        # Not sure if I am gonnaa use species but let's see
        self.species = species
        self.file_paths = file_paths 
        self.labels = labels 
        self.transforms = transforms
        self.is_test = is_test 
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        lab = self.labels[idx]
#         image = plt.imread(self.file_paths[idx])
#         if len(image.shape) == 2:
#             image = np.expand_dims(image, axis =2)
#             image = np.concatenate((image, image, image), axis=2)
        image = cv2.imread(self.file_paths[idx])
        image = image[:, :, ::-1]
        if self.transforms != None:
            res = self.transforms(image=image)
            image = res['image'].astype(np.float32)
            
        image = image.transpose(2, 0, 1) 
        image = torch.tensor(image)
        if not self.is_test:
            return {'image':image, "label" :lab, "spe" : self.species[idx]}
        else:
            return {"image":image, "label":lab}

In [ ]:
from torch.nn import Parameter
class ArcMarginProduct(nn.Module):
    r"""
    credits = https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/blob/master/src/modeling/metric_learning.py
    Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
       
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size())
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [ ]:

# # train_ds = MyDataset(file_paths = train.image, labels = train.individual_id, transforms = transforms_224)
# # train_dl = DataLoader(train_ds, batch_size=10, num_workers = 4 , pin_memory = True, drop_last = True)

# # val_ds = MyDataset(file_paths = val.image, labels = val.individual_id, transforms = transforms_224)
# # val_dl = DataLoader(val_ds,batch_size = 10, num_workers = 4 , pin_memory = True, drop_last = False)

# class AlexModel(pl.LightningModule):
#     def __init__(self):
#         super(AlexModel,self).__init__()
#         self.backbone = timm.create_model("efficientnet_b6",pretrained = True) 
#         self.linear = nn.Linear(self.backbone.classifier.out_features, 512)
#         self.softmax = nn.Softmax(dim = 0)
#         self.batch_size = BATCH_SIZE# 128 ## if TPU
#         self.arc = ArcMarginProduct( in_features = 512,
#                               out_feature s =15587 , s = 30, m = 0.3  )
    
#     def forward(self, input_image):
#         x = self.backbone(input_image)
#         x = self.linear(x)
#         return x
    
#     def training_step(self,batch,batch_idx):
#         data = batch["image"]
#         target = batch["label"]
#         spe = batch['spe']
#         logits_m = self(data)
#         lo = self.arc(logits_m,torch.tensor(spe)) 
#         lo = self.softmax(lo)
#         loss = nn.CrossEntropyLoss()(lo,torch.tensor(target, dtype = torch.long))
#         return {"loss":loss}
    
#     def validation_step(self,batch,batch_idx):
#         data = batch["image"]
#         target = batch["label"]
#         spe = batch['spe']
#         logits_m = self(data)
#         lo = self.arc(logits_m,torch.tensor(spe)) 
#         lo = self.softmax(lo)
#         loss = nn.CrossEntropyLoss()(lo,torch.tensor(target, dtype = torch.long))
#         self.log('val_loss', loss)
#         return {"val_loss":loss, "pred":logits_m}
    
#     def validation_epoch_end(self, outputs):
#         avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#         return {'avg_val_loss': avg_loss}
    
#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

#     def train_dataloader(self):
#         das = MyDataset(file_paths= train.image_id.to_numpy(), labels = train.individual_id.to_numpy(), transforms = transforms_224, species = train.species.to_numpy(), is_test = False)
#         dl = DataLoader(das, batch_size = self.batch_size, num_workers= 4 )
#         return dl

#     def val_dataloader(self):
#         dataset = MyDataset(file_paths = val.image_id.to_numpy(), labels = val.individual_id.to_numpy(), transforms = transforms_224, species = val.species.to_numpy(), is_test = False)
#         loader = DataLoader(dataset, batch_size=self.batch_size, num_workers= 4 )
#         return loader

class ArcMarginProduct(nn.Module):
    r"""
    credits = https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/blob/master/src/modeling/metric_learning.py
    Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
       
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size())
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output


class Base_mod(torch.nn.Module):
    def __init__(self):
        super(Base_mod,self).__init__()
        self.backbone = timm.create_model("t",pretrained = True) 
        self.linear = nn.Linear(self.backbone.classifier.out_features, 512)
        self.softmax = nn.Softmax(dim = 0)
    
    def forward(self, input_image):
        x = self.backbone(input_image)
        x = self.linear(x)
        return x

class AlexMod(pl.LightningModule):
    def __init__(self, base_mod, arc_margin):
        super(AlexMod, self).__init__()
        self.base = base_mod
        self.arc_face =  arc_margin
        self.softmax = nn.Softmax(dim = 0)
        self.batch_size = BATCH_SIZE
    def forward(self, input_image, spe):
        x = self.base(input_image)
        x = self.arc_face(x, spe)
        return self.softmax(x)

    
    def training_step(self,batch,batch_idx):
        data = batch["image"]
        target = batch["label"]
        spe = batch['spe']
        logits_m = self(data, spe)
        loss = nn.CrossEntropyLoss()(logits_m,torch.tensor(target, dtype = torch.long))
        return {"loss":loss}
    
    def validation_step(self,batch,batch_idx):
        data = batch["image"]
        target = batch["label"]
        spe = batch['spe']
        logits_m = self(data, spe )
        loss = nn.CrossEntropyLoss()(logits_m,torch.tensor(target, dtype = torch.long))
        return {"loss":loss}
        self.log('val_loss', loss)
        return {"val_loss":loss, "pred":logits_m}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        return {'avg_val_loss': avg_loss}
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def train_dataloader(self):
        das = MyDataset(file_paths= train.image_id.to_numpy(), labels = train.individual_id.to_numpy(), transforms = transforms_512, species = train.species.to_numpy(), is_test = False)
        dl = DataLoader(das, batch_size = self.batch_size, num_workers= 4 )
        return dl

    def val_dataloader(self):
        dataset = MyDataset(file_paths = val.image_id.to_numpy(), labels = val.individual_id.to_numpy(), transforms = transforms_512, species = val.species.to_numpy(), is_test = False)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers= 4 )
        return loader

embed_model = Base_mod()
arc_face = ArcMarginProduct( in_features = 512,
                              out_features =15587 , s = 30, m = 0.3  )
model = AlexMod(embed_model , arc_face)       
    

In [ ]:
from pytorch_lightning.loggers import WandbLogger  # newline 1
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
es = EarlyStopping(monitor="val_loss")
checkpointing = ModelCheckpoint(monitor="val_loss")
trainer = pl.Trainer(callbacks=[es,checkpointing],progress_bar_refresh_rate=20 ,gpus = 1  ,  max_epochs = 10)#, distributed_backend='ddp')
if __name__ == '__main__':
    trainer.fit(model)

In [ ]:
torch.save(embed_model.state_dict(), "./embed_model.pt")

In [ ]:
torch.save(arc_face.state_dict(), "./arc_face.pt")

In [ ]:
torch.save(model.state_dict(),"./AlexModelHap.pt")

In [ ]:

# train_targets = []
# train_embeddings = []
# for x in train_dl:
#     embeddings = mod(x['image'])
#     targets = x['label']
#     train_embeddings.append(embeddings)
#     train_targets.append(targets)
# train_embeddings = np.concatenate(train_embeddigns)
# train_targets = np.concatenate(train_targets )

In [ ]:
# torch.save(mod.state_dict(),"./AlexModelHap.pt")

# get distance vector for train images 

In [ ]:
# train_dl = mod.train_dataloader()

In [ ]:
# for x in train_dl:
#     print(x)
#     break

In [ ]:
# # creating the pytroch model AlexModel 
# class test_dataset(torch.utils.data.Dataset):
#     def __init__(self, file_paths,transforms  ):
#         # Not sure if I am gonnaa use species but let's see
#         self.file_paths = file_paths 
#         self.transforms = transforms
    
#     def __len__(self):
#         return len(self.file_paths)
    
#     def __getitem__(self,idx):
#         image = plt.imread(self.file_paths[idx])
#         if len(image.shape) == 2:
#             image = np.expand_dims(image, axis =2)
#             image = np.concatenate((image, image, image), axis=2)        
#         if self.transforms != None:
#             res = self.transforms(image=image)
#             image = res['image'].astype(np.float32)
#         image = torch.tensor(image).permute(2,0,1)
#         return {"image":image}

In [ ]:
# test_ds = test_dataset(test.image_path, transforms = transforms_224)
# test_dl = DataLoader(test_ds , batch_size = BATCH_SIZE,num_workers = 4  )

In [ ]:
# embeddings = mod(data['image'])

In [ ]:
# embeddings.shape

In [ ]:
# # from sklearn.neighbors import NearestNeighbors
# neigh = NearestNeighbors(n_neighbors=5,metric='cosine')
# neigh.fit(embeddings.detach().numpy())


In [ ]:
# neigh.kneighbors(np.array([embeddings.detach().numpy()[0]]), n_neighbors=5, return_distance=True)

In [ ]:
# test_ids = []
# test_nn_distances = []
# test_nn_idxs = []
# for filename in tqdm(test_files):
#     embeddings = get_embeddings(filename)
#     ids = get_ids(filename)
#     distances,idxs = neigh.kneighbors(embeddings, config.KNN, return_distance=True)
#     test_ids.append(ids)
#     test_nn_idxs.append(idxs)
#     test_nn_distances.append(distances)
# test_nn_distances = np.concatenate(test_nn_distances)
# test_nn_idxs = np.concatenate(test_nn_idxs)
# test_ids = np.concatenate(test_ids)

In [ ]:
# sample_submission = pd.read_csv('../input/happy-whale-and-dolphin/sample_submission.csv',index_col='image')
# print(len(test_ids),len(sample_submission))
# test_df = []
# for i in tqdm(range(len(test_ids))):
#     id_ = test_ids[i]
#     targets = train_targets[test_nn_idxs[i]]
#     distances = test_nn_distances[i]
#     subset_preds = pd.DataFrame(np.stack([targets,distances],axis=1),columns=['target','distances'])
#     subset_preds['image'] = id_
#     test_df.append(subset_preds)
# test_df = pd.concat(test_df).reset_index(drop=True)
# test_df['confidence'] = 1-test_df['distances']
# test_df = test_df.groupby(['image','target']).confidence.max().reset_index()
# test_df = test_df.sort_values('confidence',ascending=False).reset_index(drop=True)
# test_df['target'] = test_df['target'].map(target_encodings)
# test_df.to_csv('test_neighbors.csv')
# test_df.image.value_counts().value_counts()

In [ ]:
# predictions = {}
# for i,row in tqdm(test_df.iterrows()):
#     if row.image in predictions:
#         if len(predictions[row.image])==5:
#             continue
#         predictions[row.image].append(row.target)
#     elif row.confidence>best_threshold_adjusted:
#         predictions[row.image] = [row.target,'new_individual']
#     else:
#         predictions[row.image] = ['new_individual',row.target]
        
# for x in tqdm(predictions):
#     if len(predictions[x])<5:
#         remaining = [y for y in sample_list if y not in predictions]
#         predictions[x] = predictions[x]+remaining
#         predictions[x] = predictions[x][:5]
#     predictions[x] = ' '.join(predictions[x])
    
# predictions = pd.Series(predictions).reset_index()
# predictions.columns = ['image','predictions']
# predictions.to_csv('submission.csv',index=False)
# predictions.head()

In [ ]:
# plt.spy(neigh.kneighbors_graph(embeddings.detach().numpy(),n_neighbors=5))
# plt.show()